In [52]:
import requests
import numpy as np
import pandas as pd

In [46]:
years = [y for y in range(2006, 2026)]
for year in years:
    url = f'https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year={year}'
    print(url)

https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2006
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2007
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2008
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2009
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2010
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2011
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2012
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2013
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2014
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2015
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2016
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2017
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2018
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2019
https:

miliseconds -> timenow

In [47]:
from datetime import datetime

dt = datetime.fromtimestamp(1139200200000 // 1000)
dt

datetime.datetime(2006, 2, 6, 13, 30)

In [43]:
url = 'https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2023'

params = {"year": 2023}
headers = {"User-Agent": "Mozilla/5.0"}  # 해외 IP 차단 우회에 도움될 수도 있음

response = requests.get(url, params=params, headers=headers)
print(response.status_code)
print(response.text)

403
<HTML><HEAD>
<TITLE>Access Denied</TITLE>
</HEAD><BODY>
<H1>Access Denied</H1>
 
You don't have permission to access "http&#58;&#47;&#47;www&#46;fire&#46;ca&#46;gov&#47;api&#47;sitecore&#47;Incident&#47;GetFiresByYear&#63;" on this server.<P>
Reference&#32;&#35;18&#46;2dad2c17&#46;1756304453&#46;16aec81a
<P>https&#58;&#47;&#47;errors&#46;edgesuite&#46;net&#47;18&#46;2dad2c17&#46;1756304453&#46;16aec81a</P>
</BODY>
</HTML>



## API로 데이터 불러오기

내 코드

In [ ]:
response = requests.get(url)
fire = pd.DataFrame()
years = [year for year in range(2006, 2026)]
headers = {"User-Agent": "Mozilla/5.0"}

for idx, year in enumerate(years):
    url = f'https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year={year}'
    response = requests.get(url, headers=headers)
    data = response.json()
    cols = data[0].keys()
    data_array = np.array([[value for value in data[i].values()] for i in range(len(data))])
    if year == years[0]:
        fire = pd.DataFrame(data_array, columns=cols)
    else:
        new = pd.DataFrame(data_array, columns=cols)
        fire = pd.concat([fire, new], ignore_index=True)
fire

In [ ]:
headers = {"User-Agent": "Mozilla/5.0"}

#singleFire = pd.DataFrame()
for url in tqdm(fire['Url'][697:]):
    url2 = f'https://www.fire.ca.gov/api/sitecore/Incident/GetSingleFire?IncidentUrl={url}'
    response = requests.get(url2, headers=headers)
    data = response.json()
    cols = data.keys()
    data_array = [[value for value in data.values()]]
    new = pd.DataFrame(data_array, columns=cols)
    singleFire = pd.concat([singleFire, new], ignore_index=True)

개선된 코드 (gpt)

In [ ]:
headers = {"User-Agent": "Mozilla/5.0"}

all_fires = []

for url in tqdm(fire['Url'], desc="Fetching fire details"):
    url2 = f'https://www.fire.ca.gov/api/sitecore/Incident/GetSingleFire?IncidentUrl={url}'
    response = requests.get(url2, headers=headers)

    if response.status_code != 200:
        print(f"⚠️ 요청 실패 ({response.status_code}) - {url}")
        continue

    try:
        data = response.json()
        if not data:
            print(f"ℹ️ 데이터 없음 - {url}")
            continue

        # JSON dict → DataFrame 한 줄
        df = pd.DataFrame([data])
        all_fires.append(df)

    except ValueError:
        print(f"⚠️ JSON 파싱 실패 - {url}")
        print(response.text[:200])  # 앞부분만 확인

# 모든 데이터 합치기
if all_fires:
    singleFire = pd.concat(all_fires, ignore_index=True)
    print(f"✅ 총 {len(singleFire)} 건의 상세 데이터 수집 완료")
else:
    singleFire = pd.DataFrame()
    print("❌ 수집된 데이터 없음")

singleFire.head()


In [ ]:
import requests
import pandas as pd

years = range(2006, 2026)
headers = {"User-Agent": "Mozilla/5.0"}

all_data = []  # 데이터를 모아두는 리스트

for year in years:
    url = f'https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year={year}'
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"⚠️ {year} 요청 실패: {response.status_code}")
        continue

    try:
        data = response.json()
        if not data:  # 데이터가 비어있는 경우
            print(f"ℹ️ {year} 데이터 없음")
            continue

        df = pd.DataFrame(data)
        df["year"] = year  # 출처 구분을 위해 연도 컬럼 추가
        all_data.append(df)

    except ValueError:
        print(f"⚠️ {year} JSON 파싱 실패")
        print(response.text[:200])  # 앞부분만 확인

# 모든 연도 데이터 합치기
if all_data:
    fire = pd.concat(all_data, ignore_index=True)
    print(f"✅ 총 {len(fire)}건 데이터 수집 완료")
else:
    fire = pd.DataFrame()
    print("❌ 수집된 데이터 없음")

fire.head()


In [ ]:
headers = {"User-Agent": "Mozilla/5.0"}

all_fires = []

for url in tqdm(fire['Url'], desc="Fetching fire details"):
    url2 = f'https://www.fire.ca.gov/api/sitecore/Incident/GetSingleFire?IncidentUrl={url}'
    response = requests.get(url2, headers=headers)

    if response.status_code != 200:
        print(f"⚠️ 요청 실패 ({response.status_code}) - {url}")
        continue

    try:
        data = response.json()
        if not data:
            print(f"ℹ️ 데이터 없음 - {url}")
            continue

        # JSON dict → DataFrame 한 줄
        df = pd.DataFrame([data])
        all_fires.append(df)

    except ValueError:
        print(f"⚠️ JSON 파싱 실패 - {url}")
        print(response.text[:200])  # 앞부분만 확인

# 모든 데이터 합치기
if all_fires:
    singleFire = pd.concat(all_fires, ignore_index=True)
    print(f"✅ 총 {len(singleFire)} 건의 상세 데이터 수집 완료")
else:
    singleFire = pd.DataFrame()
    print("❌ 수집된 데이터 없음")

singleFire.head()


## Start

In [54]:
fire = pd.read_excel(r'D:\vscode\RhombusPower\fire.xlsx')
singleFire = pd.read_excel(r'D:\vscode\RhombusPower\singleFire.xlsx')
display(fire)
display(singleFire)

,Name,Updated,Started,AdminUnit,County,Location,AcresBurned,PercentContained,Longitude,Latitude,Type,UniqueId,Url,StartedDateOnly,IsActive,ExternalUrl
0,Sierra Fire,2006-02-12,2006-02-06,USFS/Orange County Fire,"Orange,Riverside",Corona/Orange,10584.0,100,0.000000,0.000000,NaN,7bc3e973-7c13-4e62-926a-c6d25d4fa426,/incidents/2006/2/6/sierra-fire/,NaN,False,NaN
1,Arrastre Fire,2006-06-09,2006-06-09,San Bernardino National Forest,San Bernardino,6 miles east of Big Bear City,361.0,100,0.000000,0.000000,NaN,8e7f4ac2-1628-4263-97d9-3c8d9da126e3,/incidents/2006/6/9/arrastre-fire/,NaN,False,NaN
2,Plunge Fire,2006-06-19,2006-01-23,San Bernardino National Forest,San Bernardino,2-3 miles south of Running Spring,485.0,100,0.000000,0.000000,NaN,16d75336-a2e0-4dc0-b894-873e80e3db9f,/incidents/2006/1/23/plunge-fire/,NaN,False,NaN
3,Perkins Fire,2006-06-25,2006-06-19,Los Padres National Forest,Santa Barbara,South of the community of New,15043.0,100,0.000000,0.000000,NaN,9f2d8497-ffff-4d8e-a0ac-38a340fef8ab,/incidents/2006/6/19/perkins-fire/,NaN,False,NaN
4,Del Puerto Fire,2003-07-24,2003-07-20,CDF Santa Clara Unit,Stanislaus,Del Puerto Canyon Road/MP 16,5909.0,100,0.000000,0.000000,NaN,472b6bb1-1cf3-4157-810b-6f298ba3f938,/incidents/2006/7/20/del-puerto-fire/,NaN,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4008,Butter Fire,2025-08-26,2025-08-26,Bureau of Land Management - California Desert ...,Kern,"Butterbredt Canyon Road, East of Pacific Crest...",48.0,0,-118.147758,35.464035,Wildfire,aa24ecdd-5b7c-4ef2-8423-dc88687c5965,/incidents/2025/8/26/butter-fire/,NaN,True,NaN
4009,Grade Fire,2025-08-27,2025-08-26,CAL FIRE Siskiyou Unit,Siskiyou,"West Lava Rim Road, Pondosa",10.0,15,-121.590689,41.217878,Wildfire,eb382b3b-d965-4bae-a906-c1bde34d4ec5,/incidents/2025/8/26/grade-fire/,NaN,True,NaN
4010,3-27 Fire,2025-08-26,2025-08-26,CAL FIRE Lassen-Modoc Unit,Lassen,"Southeast of Pumpkin Center, Bieber",24.0,0,-121.057196,41.088854,Wildfire,270ab2cc-1c68-4ffc-a7b2-5990bebe1de6,/incidents/2025/8/26/3-27-fire/,NaN,True,NaN
4011,H-10 Fire,2025-08-26,2025-08-26,Lassen National Forest,Shasta,"Forest Road 36N18 & Butte Creek, Hat Creek",10.6,0,-121.330697,40.737144,Wildfire,48a9cb94-d7ee-482e-a4fe-274f5b20d1ac,/incidents/2025/8/26/h-10-fire/,NaN,True,NaN


,Name,Final,Updated,Started,AdminUnit,AdminUnitUrl,County,Counties,Location,AcresBurned,...,Agencies,UnifiedCommandAgencies,mapZoom,AdditionalInfo,IncidentCameras,CameraString,Photos,MapUrl3D,DemographicsReportUrl,AdditionalIncidentMapsUrl
0,Sierra Fire,True,/Date(1139767200000)/,/Date(1139200200000)/,USFS/Orange County Fire,NaN,NaN,"['Orange', 'Riverside']",Corona/Orange,10584.0,...,"[{'Name': 'USFS/Orange County Fire', 'AgencyUR...",[],12,[],NaN,NaN,[],NaN,NaN,NaN
1,Arrastre Fire,True,/Date(1149867180000)/,/Date(1149857460000)/,San Bernardino National Forest,NaN,NaN,['San Bernardino'],6 miles east of Big Bear City,361.0,...,"[{'Name': 'San Bernardino National Forest', 'A...",[],12,[],NaN,NaN,[],NaN,NaN,NaN
2,Plunge Fire,True,/Date(1150731480000)/,/Date(1138019520000)/,San Bernardino National Forest,NaN,NaN,['San Bernardino'],2-3 miles south of Running Spring,485.0,...,"[{'Name': 'San Bernardino National Forest', 'A...",[],12,[],NaN,NaN,[],NaN,NaN,NaN
3,Perkins Fire,True,/Date(1151262000000)/,/Date(1150730280000)/,Los Padres National Forest,NaN,NaN,['Santa Barbara'],South of the community of New,15043.0,...,"[{'Name': 'Los Padres National Forest', 'Agenc...",[],12,[],NaN,NaN,[],NaN,NaN,NaN
4,Del Puerto Fire,True,/Date(1059030000000)/,/Date(1058695680000)/,CDF Santa Clara Unit,NaN,NaN,['Stanislaus'],Del Puerto Canyon Road/MP 16,5909.0,...,"[{'Name': 'CDF Santa Clara Unit', 'AgencyURL':...",[],12,[],NaN,NaN,[],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4008,Butter Fire,False,/Date(1756241505340)/,/Date(1756207966000)/,Bureau of Land Management - California Desert ...,NaN,NaN,['Kern'],"Butterbredt Canyon Road, East of Pacific Crest...",48.0,...,[],[],12,[],NaN,NaN,[],NaN,NaN,NaN
4009,Grade Fire,False,/Date(1756275093127)/,/Date(1756218897000)/,CAL FIRE Siskiyou Unit,NaN,NaN,['Siskiyou'],"West Lava Rim Road, Pondosa",10.0,...,[],[],12,[],NaN,NaN,[],NaN,NaN,NaN
4010,3-27 Fire,False,/Date(1756245737990)/,/Date(1756221842000)/,CAL FIRE Lassen-Modoc Unit,NaN,NaN,['Lassen'],"Southeast of Pumpkin Center, Bieber",24.0,...,[],[],12,[],NaN,NaN,[],NaN,NaN,NaN
4011,H-10 Fire,False,/Date(1756241380080)/,/Date(1756202750000)/,Lassen National Forest,NaN,NaN,['Shasta'],"Forest Road 36N18 & Butte Creek, Hat Creek",10.6,...,[],[],12,[],NaN,NaN,[],NaN,NaN,NaN
